In [ ]:
import os, sys, platform, random, json, time
import numpy as np
import torch
import faiss
import langchain
import langgraph
import sentence_transformers
import transformers


In [ ]:
!pip install langgraph langchain langchain-community langchain-text-splitters faiss-cpu sentence-transformers transformers accelerate pypdf datasets ipykernel

### Beállítások (Config)

Adatforrások, modellek, RAG-paraméterek és hiperparaméterek a reprodukálhatóság érdekében.

In [ ]:
from pathlib import Path

USE_RAG = True

PDF_URLS = ["https://arxiv.org/pdf/1706.03762.pdf", 
"https://www.medrxiv.org/content/10.1101/2024.08.10.24311686v1.full.pdf",
"https://arxiv.org/pdf/2107.09559",]
PDF_PATHS = ["./data/attention_is_all_you_need.pdf",
"./data/brainagenext.pdf",
"./data/synthseg.pdf",]

#MODELS
EMBEDDING_MODEL_NAME = "intfloat/e5-small-v2"                 
RERANKER_MODEL_NAME = "cross-encoder/ms-marco-MiniLM-L-6-v2" 
LLM_MODEL_NAME = "Qwen/Qwen2.5-3B-Instruct" # "microsoft/phi-1_5"   

#RAG
CHUNK_SIZE = 800
CHUNK_OVERLAP = 120
TOP_K = 8                
RERANK_TOP_K = 5          
ENABLE_RERANKER = True   

#HYPERPARAMS
MAX_STEPS_PER_SUBTASK = 2  # num of retries
MIN_REFLECT_SCORE = 0.6    # retry decision
MAX_NEW_TOKENS = 300
TEMPERATURE = 0.1

### Data Loading

PDF-ek letöltése, beolvasása (PyPDFLoader) és szeletelése (RecursiveCharacterTextSplitter). `chunk_id` metaadat a későbbi citációkhoz.

In [4]:
from typing import List
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
import urllib.request


def download_data():
    Path("./data").mkdir(parents=True, exist_ok=True)
    for i in range(len(PDF_URLS)):
        out_path = Path(PDF_PATHS[i])
        if not out_path.exists():
            print(f"Downloading {PDF_PATHS[i]} PDF...")
            urllib.request.urlretrieve(PDF_URLS[i], str(out_path))
            print("Saved to", out_path)
    print(f"Downloaded {len(PDF_PATHS)} scientific papers")


def load_documents() -> List[Document]:
    docs: List[Document] = []
    for p in PDF_PATHS:
        loader = PyPDFLoader(p)
        docs.extend(loader.load())

    print(f"Loaded {len(docs)} raw documents")
    return docs

def chunk_documents(docs: List[Document]) -> List[Document]:
    splitter = RecursiveCharacterTextSplitter(chunk_size=CHUNK_SIZE, chunk_overlap=CHUNK_OVERLAP)
    splits = splitter.split_documents(docs)
    # chunk_id to CITE
    for i, d in enumerate(splits):
        d.metadata = dict(d.metadata)
        d.metadata["chunk_id"] = i
    print(f"Chunked into {len(splits)} chunks")
    return splits

download_data()
raw_docs = load_documents()
chunks = chunk_documents(raw_docs)

Saved to data/attention_is_all_you_need.pdf
Saved to data/brainagenext.pdf
Saved to data/synthseg.pdf
Downloaded 3 scientific papers
Loaded 58 raw documents
Chunked into 305 chunks


### Embedding and Vector database

E5 beágyazások + FAISS index; a retriever `k=TOP_K` paraméterrel dolgozik a hasonlósági kereséshez.

In [ ]:
from sentence_transformers import SentenceTransformer
from langchain.embeddings.base import Embeddings
from langchain_community.vectorstores import FAISS

class E5Embeddings(Embeddings):
    def __init__(self, model_name: str):
        self.model = SentenceTransformer(model_name)
        
    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        return self.model.encode([f"passage: {t}" for t in texts], normalize_embeddings=True).tolist()

    def embed_query(self, text: str) -> List[float]:
        return self.model.encode([f"query: {text}"], normalize_embeddings=True)[0].tolist()

embeddings = E5Embeddings(EMBEDDING_MODEL_NAME)

# Vector store
vectorstore = FAISS.from_documents(chunks, embeddings)
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": TOP_K})

print("Vector store ready.")

### Rerank

CrossEncoder (ms-marco-MiniLM-L-6-v2) a top találatok újrapontozására a esetleges minőség javításra.

In [ ]:
import numpy as np
from sentence_transformers import CrossEncoder

cross_encoder = CrossEncoder(RERANKER_MODEL_NAME) if ENABLE_RERANKER else None

def rerank(query: str, docs: List[Document], top_k: int = RERANK_TOP_K) -> List[Document]:

    pairs = [(query, d.page_content[:512]) for d in docs] 
    scores = cross_encoder.predict(pairs)
    idx = np.argsort(scores)[::-1]  
    return [docs[i] for i in idx[:top_k]]

### LLM Model Init
Transformers pipeline helyi, cserélhető modellhez; `max_new_tokens` és `temperature` a válasz stílusát/hosszát szabályozza.

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

def build_llm(model_name: str, max_new_tokens: int = MAX_NEW_TOKENS, temperature: float = TEMPERATURE):
    try:
        tok = AutoTokenizer.from_pretrained(model_name)
        model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto")  
        gen = pipeline("text-generation", model=model, tokenizer=tok, max_new_tokens=max_new_tokens, do_sample=(temperature>0), temperature=temperature)
        def llm(prompt: str) -> str:
            out = gen(prompt, return_full_text=False)[0]["generated_text"]
            return out.strip()  
        print(f"Loaded LLM: {model_name}")
        return llm
    except Exception as e:
        print("Failed to load LLM. Error:", e)

llm = build_llm(LLM_MODEL_NAME)

### Agent Node Components
- `plan`: a felhasználói kérdést ≤4 részfeladatra bontja.
- `retrieve`: FAISS-ból kontextus visszakeresése (majd opcionális rerank).
- `synthesize`: LLM válasz a kontextusból, kötelező inline citációkkal [i].
- `reflect`: heurisztikus minőségellenőrzés (hossz + citációk).
- `finalize`: részválaszok koherens összevonása + „Sources” szekció.

In [ ]:
from typing import TypedDict, Optional, Dict, Any
from langgraph.graph import StateGraph, END
import json
import re



class AgentState(TypedDict):
    question: str
    plan: list[str]                   
    step_idx: int                     
    retrieved: list[Document]        
    partial_answers: list[str]       
    citations: list[list[Dict[str, Any]]] 
    final_answer: Optional[str]
    reflect_score: float
    history: list[Dict[str, Any]]     


def plan_node(state: AgentState) -> AgentState:
    q = state["question"]
    prompt = f"""You are a planning agent. Split the user's question into up to 4 logical subtasks as a JSON list.

    Question: {q}

    Output example: ["Define the concept", "Explain the background", "Give practical examples"]"""
    try:
        out = llm(prompt)
        plan = json.loads(out) if out.strip().startswith("[") else [q]
    except Exception:
        plan = [q]
    state["plan"] = plan[:4] if plan else [q]
    state["step_idx"] = 0
    state["partial_answers"] = []
    state["citations"] = []
    state["history"] = state.get("history", []) + [{"role": "system", "content": f"Plan: {state['plan']}"}]
    return state

def retrieve_node(state: AgentState) -> AgentState:
    step = state["plan"][state["step_idx"]]
    docs = retriever.invoke(step)
    docs = rerank(step, docs, top_k=RERANK_TOP_K) if ENABLE_RERANKER else docs[:RERANK_TOP_K]
    state["retrieved"] = docs
    state["history"].append({"role": "tool", "content": f"Retrieved {len(docs)} docs for step '{step}'"})
    return state


def synthesize_node(state: AgentState) -> AgentState:
    step = state["plan"][state["step_idx"]]
    ctx_docs = state["retrieved"][:4]
    context = "\n\n".join([f"[{i}] {d.page_content[:1200]}" for i, d in enumerate(ctx_docs)])
    
    prompt = f"""You are answering a scientific question using provided sources.

    Subtask: {step}

    Sources:
    {context}

    Instructions:
    1. Write a clear, direct answer to the subtask
    2. Include inline citations [0], [1], [2] next to every claim
    3. Do NOT just list the sources - write actual explanatory text
    4. Keep it concise but informative

    Answer:"""
    answer = llm(prompt)
    cited_ids = sorted(set(int(x) for x in re.findall(r"\[(\d+)\]", answer) if x.isdigit() and int(x) < len(ctx_docs)))
    citations = [
        {
            "chunk_id": ctx_docs[i].metadata.get("chunk_id"),
            "metadata": ctx_docs[i].metadata
        }
        for i in cited_ids]
    state["partial_answers"].append(answer)
    state["citations"].append(citations)
    state["history"].append({"role": "assistant", "content": f"Draft answer for '{step}'"})
    return state

def reflect_node(state: AgentState) -> AgentState:
    ans = state["partial_answers"][-1] if state["partial_answers"] else ""
    has_cite = "[" in ans and "]" in ans
    length_ok = len(ans.split()) > 25
    score = 0.5 * (1.0 if has_cite else 0.0) + 0.5 * (1.0 if length_ok else 0.0)
    state["reflect_score"] = float(score)
    state["history"].append({"role": "system", "content": f"Reflect score: {score:.2f}"})
    return state

def finalize_node(state: AgentState) -> AgentState:
    combined = "\n\n".join([f"Subtask {i+1}: {p}\nAnswer:\n{a}" for i, (p, a) in enumerate(zip(state["plan"], state["partial_answers"]))])
    prompt = f"""Merge the sub-answers into a coherent final answer. Preserve all inline citations like [0], [1].
        
    Sub-answers:
    {combined}

    Write a clear, well-structured final answer that preserves all citations:"""
        
    final = llm(prompt)
    state["final_answer"] = final
    
    all_citations = {}
    for subtask_cites in state["citations"]:
        for cite in subtask_cites:
            chunk_id = cite["chunk_id"]
            if chunk_id is not None and chunk_id not in all_citations:
                all_citations[chunk_id] = cite["metadata"]
    
    if all_citations:
        citation_text = "\n\nSources:\n"
        for i, (chunk_id, metadata) in enumerate(all_citations.items()):
            source = metadata.get('source', 'Unknown')
            page = metadata.get('page', 'N/A')
            citation_text += f"[{i}] {source}, page {page}\n"
        state["final_answer"] = final + citation_text
    
    state["history"].append({"role": "assistant", "content": "Final answer synthesized."})
    return state

def should_retry(state: AgentState) -> bool:
    # Retry if quality is low and we haven't exhausted attempts
    attempts_this_step = sum(1 for h in state["history"] if h.get("role")=="assistant" and "Draft answer" in h.get("content",""))
    return (state["reflect_score"] < MIN_REFLECT_SCORE) and (attempts_this_step < MAX_STEPS_PER_SUBTASK)

### LangGraph Architecture
```mermaid
flowchart LR
  plan --> retrieve --> synthesize --> reflect
  reflect -- retry (loop back) --> retrieve
  reflect -- next subtask (loop back) --> retrieve
  reflect -- finalize --> finalize --> OUT((END))
  ```

In [ ]:
from langgraph.graph import StateGraph

graph = StateGraph(AgentState)
graph.add_node("plan", plan_node)
graph.add_node("retrieve", retrieve_node)
graph.add_node("synthesize", synthesize_node)
graph.add_node("reflect", reflect_node)
graph.add_node("finalize", finalize_node)

graph.set_entry_point("plan")
graph.add_edge("plan", "retrieve")
graph.add_edge("retrieve", "synthesize")
graph.add_edge("synthesize", "reflect")

def route_after_reflect(state: AgentState):
    if should_retry(state):
        return "retrieve"
    if state["step_idx"] + 1 < len(state["plan"]):
        state["step_idx"] += 1
        return "retrieve"
    else:
        return "finalize"

graph.add_conditional_edges(
    "reflect",
    route_after_reflect,
    {
        "retrieve": "retrieve",
        "finalize": "finalize"
    }
)

graph.add_edge("finalize", END)
app = graph.compile()
print("Graph compiled.")

Graph compiled.


### Agent Inference
Három kérdés feltevés a három különböző tudományos cikkből, amelyek a vektor tudástárba vannak betöltve.

In [10]:
def run_agent(question: str):
    init: AgentState = {
        "question": question,
        "plan": [],
        "step_idx": 0,
        "retrieved": [],
        "partial_answers": [],
        "citations": [],
        "final_answer": None,
        "reflect_score": 0.0,
        "history": []
    }
    out = app.invoke(init)
    return out["final_answer"], out

queries = ["Provide a concise summary of the core ideas of the Transformer architecture. Include citations.",
"Provide a concise overview of Synthseg and why it is considered a novelty in practice. Include citations.",
"Provide a concise description of BrainAgeNext, including its purpose and applications. Include citations."]
for query in queries:
    final_answer, state = run_agent(query)
    print(10*"=")
    print(f"Answer for {query}:")
    print(final_answer)

Answer for Provide a concise summary of the core ideas of the Transformer architecture. Include citations.:
The Transformer architecture, introduced as a novel approach to sequence transduction, comprises stacked self-attention mechanisms and point-wise, fully connected layers for both the encoder and decoder. Each layer includes a multi-head self-attention mechanism followed by a position-wise feed-forward network, both wrapped in residual connections with layer normalization. This design facilitates efficient parallel processing of sequences without the need for recurrence. According to [0], the architecture is composed of six identical layers, each containing these two sub-layers. The Transformer's design offers advantages over traditional recurrent neural networks, including faster training times and improved performance on translation tasks [1]. Various variations of the Transformer have been explored, showcasing its adaptability and effectiveness across diverse applications [2]. 

### RAG Query Sanity Check
Gyors „sanity check” a RAG minőségének ellenőrzésére: Ugyanarra a 3 különböző kontextusu kérdésre kilistázza a FAISS és a rerank algoritmus (CrossEncoder). Ez által láthatjuk a relevanciát és hangolhatjuk a későbbiekben a beállításokat (TOP_K, RERANK_TOP_K, CHUNK_SIZE/OVERLAP) a végső válaszgenerálás nélkül.


In [12]:
queries = ["Provide a concise summary of the core ideas of the Transformer architecture. Include citations.",
"Provide a concise overview of Synthseg and why it is considered a novelty in practice. Include citations.",
"Provide a concise description of BrainAgeNext, including its purpose and applications. Include citations."]

for query in queries:
    print(10*"=")
    print("For query:", query)
    docs = retriever.invoke(query)  
    docs = rerank(query, docs) if ENABLE_RERANKER else docs[:5]
    for i, d in enumerate(docs):
        print(f"Doc {i} | chunk_id={d.metadata.get('chunk_id')}\n{d.page_content[:300]}\n---\n")

For query: Provide a concise summary of the core ideas of the Transformer architecture. Include citations.
Doc 0 | chunk_id=12
Figure 1: The Transformer - model architecture.
The Transformer follows this overall architecture using stacked self-attention and point-wise, fully
connected layers for both the encoder and decoder, shown in the left and right halves of Figure 1,
respectively.
3.1 Encoder and Decoder Stacks
Encoder
---

Doc 1 | chunk_id=47
7 Conclusion
In this work, we presented the Transformer, the first sequence transduction model based entirely on
attention, replacing the recurrent layers most commonly used in encoder-decoder architectures with
multi-headed self-attention.
For translation tasks, the Transformer can be trained signi
---

Doc 2 | chunk_id=40
Table 3: Variations on the Transformer architecture. Unlisted values are identical to those of the base
model. All metrics are on the English-to-German translation development set, newstest2013. Listed
perplexities are pe

## Rövid Dokumentáció és Értékelés

### Notebook
A notebook lépésről lépésre futtatható. Config → Adatbetöltés és szeletelés → Beágyazások és vektortár → Rerankelés → LLM betöltés → LangGraph Agent pipeline (plan → retrieve → synthesize → reflect → finalize) → Kérdések futtatása → RAG query megtekintése.


### Technológiák
- **PyPDFLoader**: Megbízható PDF feldolgozás, a `langchain_community` ökoszisztémába illeszkedik; a metaadatok (pl. oldal) automatikusan kezelhetők.
- **RecursiveCharacterTextSplitter**: `CHUNK_SIZE=800`, `CHUNK_OVERLAP=120` balansz a kontextus-megőrzés és a token-költség/latencia között; stabil általános beállítás tudományos PDF-ekhez.
- **Embedding: intfloat/e5-small-v2**: E5-széria, főleg erős zero-shot információkeresésben; a „query:”/„passage:” prefix segíti a lekérdező–passage illesztést. Kis modell → gyors CPU/MPS futás.
- **FAISS CPU**: Gyors, lokális, könnyen fenntartható lokális tárban, általánosságban jó kiindulás kisebb korpuszon. `TOP_K=8` a visszakeresésnél.
- **Rerank: cross-encoder/ms-marco-MiniLM-L-6-v2**: A retrieval pontosságnövelése érdekében raktam bele a pipelineba, hogy a legjobb 5 találatot újra rangsorolja. Egy külön,light wegiht modell, amely közvetlenül a (kérdés, eredmény) párokat pontozza.
- **LLM: Qwen/Qwen2.5-3B-Instruct, transformers**: Kompetens, de még pont PC-n helyben futtatható; `device_map="auto"` MPS/GPU támogatással, `temperature` és `max_new_tokens` szabályozza a stílust és hosszt.



### Teljesítmény és tesztelés
A minőségi teszteléshez érdemes egy kézzel készített aranykészletet összeállítani változatos, valós kérdés–válasz párokkal és a hozzájuk tartozó releváns source sectionökkel, mert ez ad objektív alapot tud adni a méréshez. Ezenkívül automatizált metrikákkal kell mérni a retrievel és a válaszok minőségét, például Recall@K, MRR, groundedness, factuality, relevancia, teljesség és helyes citáció, szükség esetén LLM-es bírálóval, amelyet időnként emberi ellenőrzéssel kalibrálunk. Hasznos egy egyszerű RAG-eval pipeline, amely a retrieve–rerank–generate lépések után kiszámítja a mutatókat, riportot készít és felsorolja a hibás eseteket, hogy látható legyen, hol romlik a rendszer. Érdemes külön stresszteszteket futtatni különböző kérdéstípusokra és edge case-ekre. Fontos az is hogy rosszabb eredményekhez logoljunk kérdést, találatokat, pontszámokat, promptot és választ, hogy később könnyebb legyen a hiba elkülönítése a retrieval és a generálás közül. 

### Javasolt további fejlesztések
Egy fontos upgrade a rendszer továbbfejlesztéséhez lenne fennmaradásának biztosítása, ami a FAISS-index mentését és újratöltését, valamint a beágyazások cache-elését a gyorsabb kiszolgálás érdekében. Továbbá a keresés minősége hibrid megközelítéssel is talán javítható, a lexikális BM25 és sűrű vektoros módszerek kombinálásával, kiegészítve MMR-rel és multi query expansionnel a lekérdezések sokszínűségére. A rangsorolásnál mérlegelhető egy nagyobb modell használata a jobb minőségért, vagy akár teljesen kikapcsolni a nagyobb sebességért, mindkét esetben érdemes a RERANK_TOP_K paraméter finomhangolása. Az LLM inferencia gyorsítható bitkvantálással, kisebb modellek, például Qwen2.5 1.5B bevetésével, vagy clusteren való távoli inferenciával. Az agent pipeline is továbbfejleszthető webes kereséssel vagy egyéb ;POST' típusú tool-ok hozzáadásával. A kontextuskezelés javítható hosszabb passzusok kinyerésével, extractive vagy összegző megközelítéssel, így több releváns forrás férhet be a promptba. Emellett célszerű a promptot és a főbb hiperparamétereket, így a CHUNK_SIZE, a CHUNK_OVERLAP, a TOP_K, a MAX_NEW_TOKENS és a TEMPERATURE szisztematikusan grid vagy bayesi optimalizálással hangolni. Az explainability érdekében hasznos részletes metrikákat és trace-eket gyűjteni. Végül érdemes egy quality control keretet is természetesen kialakítani, amely automatizált RAG értékelési pipeline-t tartalmaz.